<a href="https://colab.research.google.com/github/XiaoLiang28/Xiao/blob/main/Fall23_PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset 1: https://www.nj.gov/education/doedata/enr/ (2020-2021 Enrollment District Reported Data)

Dateset 2: https://www.greatschools.org/schools/districts/New_Jersey/NJ/

Dateset 3: https://rc.doe.state.nj.us/download (2020-2021 Performance Reports Database - Excel, ELAMathPerformanceTrends)

# Set Up


In [1]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

# Dataset 1

In [2]:
df1=pd.read_csv("https://github.com/XiaoLiang28/Xiao/raw/main/enrollment_2021.csv")

In [3]:
df1

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,2020-2021 Enrollment Data - District Counts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,...,Tenth Grade,Eleventh Grade,Twelfth Grade,Ungraded,%Free Lunch,%Reduced Lunch,%English Learners,%Migrant,%Military,%Homeless
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,...,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,...,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,...,399,355,362,160,48.6,9.6,0,0,2.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,...,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,...,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,...,0,0,0,0,73.1,13.9,6.4,0,0,0
671,80,Charters,8140,The Village Charter School,363,4,1.1,345,95,14,...,0,0,0,0,64.5,5.5,0,0,0,0


In [4]:
df1.drop([0, 1, 672],inplace=True)

In [5]:
df1

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,...,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,...,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,...,399,355,362,160,48.6,9.6,0,0,2.5,0.6
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,...,3,5,13,244,39.5,5.9,0.3,0,0.3,1
6,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,...,0,0,0,12,33.8,8.3,3.4,0,2.3,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,...,33,34,39,0,61.3,10,4,0,0,4.8
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,...,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,...,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,...,0,0,0,0,73.1,13.9,6.4,0,0,0


In [6]:
pd.set_option('display.max_columns', None)

In [7]:
df1

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,0,48,0,83,94,76,107,88,84,94,94,94,0,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,44.1,1046,16.2,1,0,42,0.7,102,1.6,0,340,0,379,482,429,472,440,457,473,487,473,496,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,42.1,102,6,5,0.3,9,0.5,64,3.7,0,0,0,0,0,0,0,0,0,0,0,0,437,399,355,362,160,48.6,9.6,0,0,2.5,0.6
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,35.3,11,3.6,1,0.3,5,1.6,4,1.3,0,4,0,0,0,8,4,3,6,3,8,4,1,3,5,13,244,39.5,5.9,0.3,0,0.3,1
6,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,14,48,10.8,0,0,0,0,2,0.5,0,56,0,34,38,46,39,41,34,42,58,44,0,0,0,0,12,33.8,8.3,3.4,0,2.3,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,38.3,4,1,0,0,0,0,0,0,0,0,0,16,18,20,20,22,24,44,35,39,56,33,34,39,0,61.3,10,4,0,0,4.8
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,11.4,37,15.1,0,0,0,0,17,6.9,0,0,0,28,25,29,25,25,28,30,23,32,0,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,32.3,39,8.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,116,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,15.5,1,0.1,0,0,0,0,0,0,0,119,0,73,82,89,115,106,66,76,63,69,0,0,0,0,0,73.1,13.9,6.4,0,0,0


In [8]:
df1 = df1.rename(columns={'Unnamed: 0': 'County Code', 'Unnamed: 1': 'County Name', 'Unnamed: 2': 'District Code', 'Unnamed: 3': 'District Name', 'Unnamed: 4': 'Total Enrollment', 'Unnamed: 5': 'White', 'Unnamed: 6': '%White', 'Unnamed: 7': 'Black', 'Unnamed: 8': '%Black', 'Unnamed: 9': 'Hispanic', 'Unnamed: 10': '%Hispanic', 'Unnamed: 11': 'Asian', 'Unnamed: 12': '%Asian', 'Unnamed: 13': 'Native American', 'Unnamed: 14': '%Native American', 'Unnamed: 15': 'Hawaiian Native', 'Unnamed: 16': '%Hawaiian Native', 'Unnamed: 17': 'Two or More Races', 'Unnamed 18': '%Two or More Races'})

In [9]:
df1

,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,Native American,%Native American,Hawaiian Native,%Hawaiian Native,Two or More Races,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,0,48,0,83,94,76,107,88,84,94,94,94,0,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,44.1,1046,16.2,1,0,42,0.7,102,1.6,0,340,0,379,482,429,472,440,457,473,487,473,496,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,42.1,102,6,5,0.3,9,0.5,64,3.7,0,0,0,0,0,0,0,0,0,0,0,0,437,399,355,362,160,48.6,9.6,0,0,2.5,0.6
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,35.3,11,3.6,1,0.3,5,1.6,4,1.3,0,4,0,0,0,8,4,3,6,3,8,4,1,3,5,13,244,39.5,5.9,0.3,0,0.3,1
6,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,14,48,10.8,0,0,0,0,2,0.5,0,56,0,34,38,46,39,41,34,42,58,44,0,0,0,0,12,33.8,8.3,3.4,0,2.3,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,38.3,4,1,0,0,0,0,0,0,0,0,0,16,18,20,20,22,24,44,35,39,56,33,34,39,0,61.3,10,4,0,0,4.8
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,11.4,37,15.1,0,0,0,0,17,6.9,0,0,0,28,25,29,25,25,28,30,23,32,0,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,32.3,39,8.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,116,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,15.5,1,0.1,0,0,0,0,0,0,0,119,0,73,82,89,115,106,66,76,63,69,0,0,0,0,0,73.1,13.9,6.4,0,0,0


In [10]:
df1=df1[df1['County Code']!='End of worksheet']

In [11]:
df1

,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,Native American,%Native American,Hawaiian Native,%Hawaiian Native,Two or More Races,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42,4.7,0,48,0,83,94,76,107,88,84,94,94,94,0,0,0,0,25,40.8,5.9,8.7,0,0.8,0.2
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,44.1,1046,16.2,1,0,42,0.7,102,1.6,0,340,0,379,482,429,472,440,457,473,487,473,496,462,405,369,286,81.5,3.6,16.1,0,0,2.2
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,42.1,102,6,5,0.3,9,0.5,64,3.7,0,0,0,0,0,0,0,0,0,0,0,0,437,399,355,362,160,48.6,9.6,0,0,2.5,0.6
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,35.3,11,3.6,1,0.3,5,1.6,4,1.3,0,4,0,0,0,8,4,3,6,3,8,4,1,3,5,13,244,39.5,5.9,0.3,0,0.3,1
6,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,14,48,10.8,0,0,0,0,2,0.5,0,56,0,34,38,46,39,41,34,42,58,44,0,0,0,0,12,33.8,8.3,3.4,0,2.3,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,38.3,4,1,0,0,0,0,0,0,0,0,0,16,18,20,20,22,24,44,35,39,56,33,34,39,0,61.3,10,4,0,0,4.8
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,11.4,37,15.1,0,0,0,0,17,6.9,0,0,0,28,25,29,25,25,28,30,23,32,0,0,0,0,0,9.8,1.6,1.2,0,0,0
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,32.3,39,8.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,116,118,105,100,0,53.3,9.8,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,15.5,1,0.1,0,0,0,0,0,0,0,119,0,73,82,89,115,106,66,76,63,69,0,0,0,0,0,73.1,13.9,6.4,0,0,0


In [12]:
df1.drop(['Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41'],axis=1, inplace=True)

In [13]:
df1

,County Code,County Name,District Code,District Name,Total Enrollment,White,%White,Black,%Black,Hispanic,%Hispanic,Asian,%Asian,Native American,%Native American,Hawaiian Native,%Hawaiian Native,Two or More Races
2,01,Atlantic,0010,Absecon Public Schools District,887,447,50.4,133,15,208,23.4,57,6.4,0,0,0,0,42
3,01,Atlantic,0110,Atlantic City School District,6450,265,4.1,2147,33.3,2847,44.1,1046,16.2,1,0,42,0.7,102
4,01,Atlantic,0120,Atlantic County Vocational School District,1713,516,30.1,296,17.3,721,42.1,102,6,5,0.3,9,0.5,64
5,01,Atlantic,0125,Atlantic County Special Services School District,306,95,31,82,26.8,108,35.3,11,3.6,1,0.3,5,1.6,4
6,01,Atlantic,0570,Brigantine Public School District,444,314,70.7,18,4.1,62,14,48,10.8,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,80,Charters,8010,Union County Teams Charter School-High School/...,400,1,0.3,242,60.5,153,38.3,4,1,0,0,0,0,0
668,80,Charters,8050,Unity Charter School,245,123,50.2,40,16.3,28,11.4,37,15.1,0,0,0,0,17
669,80,Charters,8060,University Academy Charter High School,439,18,4.1,240,54.7,142,32.3,39,8.9,0,0,0,0,0
670,80,Charters,8065,University Heights Charter School,858,0,0,724,84.4,133,15.5,1,0.1,0,0,0,0,0


In [16]:
df1[['County Name', 'District Name']][0:3]

,County Name,District Name
2,Atlantic,Absecon Public Schools District
3,Atlantic,Atlantic City School District
4,Atlantic,Atlantic County Vocational School District


In [17]:
max(df1['%White'])
print()
min(df1['%White'])

'99'

'0'

In [ ]:
df1.loc[df1.%Black>20,'price']

# Dataset 2

In [ ]:
df2=pd.read_html('https://www.greatschools.org/schools/districts/New_Jersey/NJ/',match='District name',flavor=None)[0]

In [ ]:
df2

,District name,City,County name
0,Absecon Public School District,Absecon,Atlantic County
1,Academy Charter High School,Lake Como,Monmouth County
2,Academy For Urban Leadership,Perth Amboy,Middlesex County
3,Achieve Community Charter School,Newark,Essex County
4,Achievers Early College Prep Charter School,Trenton,Mercer County
...,...,...,...
677,Woodland Park Public School District,Woodland Park,Passaic County
678,Woodland Township Board Of Education School Di...,Browns Mills,Burlington County
679,Woodlynne Boro Public School District,Woodlynne,Camden County
680,Woodstown-Pilesgrove Regional School District,Woodstown,Salem County


# Dataset 3

In [ ]:
df3=pd.read_excel('https://github.com/XiaoLiang28/Xiao/raw/main/Math%202020-2021.xlsx')

In [ ]:
df3

,CountyCode,CountyName,DistrictCode,DistrictName,SchoolYear,Subject,ParticipationRate,ProficiencyRateforFederalAccountability,AnnualTarget,MetAnnualTarget,StatewideProficiencyRateforFederalAccountability
0,01,Atlantic,0010,Absecon Public Schools District,2018-2019,ELA,98.7,50.3,56,Not Met,57.9
1,01,Atlantic,0010,Absecon Public Schools District,2018-2019,Math,98.7,43.7,44.3,Met Target†,44.5
2,01,Atlantic,0110,Atlantic City School District,2018-2019,ELA,96.6,32.7,38.9,Not Met,57.9
3,01,Atlantic,0110,Atlantic City School District,2018-2019,Math,96.9,21.8,31.9,Not Met,44.5
4,01,Atlantic,0120,Atlantic County Vocational School District,2018-2019,ELA,99.4,53.4,47.4,Met Target,57.9
...,...,...,...,...,...,...,...,...,...,...,...
1299,80,Charters,8065,University Heights Charter School,2018-2019,Math,88.4,15.3,31.3,Not Met,44.5
1300,80,Charters,8140,The Village Charter School,2018-2019,ELA,95.4,59.5,51.3,Met Target,57.9
1301,80,Charters,8140,The Village Charter School,2018-2019,Math,95.4,41.4,31.6,Met Target,44.5
1302,State,State,State,State,2018-2019,ELA,98.0,NaN,NaN,NaN,57.9
